<a href="https://colab.research.google.com/github/Ab2207/Academic-Projects-Data-Analytics/blob/main/Severity%20and%20similarity%20checker%20in%20tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np 
import string 
string.punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
import re
import unicodedata

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_data = pd.read_csv("/content/gdrive/My Drive/train.csv")
test_data = pd.read_csv("/content/gdrive/My Drive/test.csv")

In [ ]:
def tweets_cleaning(text):
    lowercase = text.lower()
    punc_removal = [char for char in lowercase if char not in string.punctuation]
    punc_removal_joined = ''.join(punc_removal)
    url_removal = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', punc_removal_joined, flags=re.MULTILINE)
    emoji_removal = url_removal.encode('ascii', 'ignore').decode('ascii')
    stopwords_removal = [word for word in emoji_removal.split() if word not in stopwords.words('english')]
    return stopwords_removal

train_data['cleaned_text'] = train_data['text'].apply(tweets_cleaning)
test_data['cleaned_text'] = test_data['text'].apply(tweets_cleaning)

In [ ]:
x_train = train_data['cleaned_text']
y_train = train_data['label']
x_test = test_data['cleaned_text']
y_test = test_data['label']

In [ ]:
x_train

0        [police, investigating, suspicious, fire, hunt...
1        [rt, andrewconstance, reminder, road, users, p...
2        [rt, mattkeanmp, regional, jobs, cheaper, powe...
3        [rt, jackbegbie, love, tram, content, andor, d...
4        [rt, pfa809, wildfire, risk, rises, plan, plac...
                               ...                        
14995                      [rt, lesshumbleteej, lmaoooooo]
14996    [rt, daandina, es, desolador, que, en, madrid,...
14997    [rt, lsaazul, los, jovconfuturo, del, centro, ...
14998    [rt, gwendavies12345, yvonner100, rishisunak, ...
14999    [rt, accessemployers, makes, persons, disabili...
Name: cleaned_text, Length: 15000, dtype: object

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, LSTM, Flatten, GlobalAveragePooling1D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [ ]:
tokenizer = Tokenizer(oov_token = "<OOV>")
tokenizer.fit_on_texts(x_train.values)

word_index = tokenizer.word_index #word tokenizing

In [ ]:
vocab_size = len(word_index) + 1
pad_pos = "post"
trunc_pos = "post"
epochs = 30
drop_rate = 0.1
input_len = 27
loss = "binary_crossentropy"
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
batch_size = 1000

In [ ]:
train_sentence_tokenizing = tokenizer.texts_to_sequences(x_train.values)
train_padding = pad_sequences(train_sentence_tokenizing, padding = pad_pos, truncating = trunc_pos)
train_padding

array([[  582,  5495, 13634, ...,     0,     0,     0],
       [    2,  2115,   882, ...,     0,     0,     0],
       [    2,  8864,  4609, ...,     0,     0,     0],
       ...,
       [    2,  3017,    27, ...,     0,     0,     0],
       [    2, 13062, 13063, ...,     0,     0,     0],
       [    2, 35315,   331, ...,     0,     0,     0]], dtype=int32)

In [ ]:
test_token = Tokenizer(oov_token = "<OOV>")
test_token.fit_on_texts(x_test.values)

test_word_index = test_token.word_index
test_sentence_tokenizing = test_token.texts_to_sequences(x_test.values)
test_padding = pad_sequences(test_sentence_tokenizing, padding = pad_pos, truncating = trunc_pos)

In [ ]:
test_padding

array([[   2, 1595,  280, ...,    0,    0,    0],
       [   2, 4414, 4415, ...,    0,    0,    0],
       [1452,  134,  749, ...,    0,    0,    0],
       ...,
       [   2,  626,  627, ...,    0,    0,    0],
       [   2,  626,  627, ...,    0,    0,    0],
       [   2,  189,  170, ...,    0,    0,    0]], dtype=int32)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padding[1]))
print(train_sentence_tokenizing[1])

rt andrewconstance reminder road users please safe around light rail almost every day seeing risky dangerous b ? ? ? ? ? ? ? ? ? ?
[2, 2115, 882, 1387, 1590, 141, 101, 385, 926, 1945, 748, 151, 71, 962, 2116, 1119, 333]


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length = train_padding.shape[1]))
#model.add(GlobalAveragePooling1D())
model.add(Bidirectional(LSTM(128)))
#model.add(Dense(32, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))


model.compile(loss = loss, optimizer = optimizer, metrics = ['accuracy'])

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 27, 128)           4520448   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 257       
Total params: 4,783,873
Trainable params: 4,783,873
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_padding, y_train, epochs = epochs, batch_size = batch_size, validation_data = (test_padding, y_test))

Epoch 1/30
15/15 [==============================] - 2s 121ms/step - loss: 0.7790 - accuracy: 0.6185 - val_loss: 0.7669 - val_accuracy: 0.5136
Epoch 2/30
15/15 [==============================] - 1s 80ms/step - loss: 0.3929 - accuracy: 0.8304 - val_loss: 0.9685 - val_accuracy: 0.5155
Epoch 3/30
15/15 [==============================] - 1s 77ms/step - loss: 0.1212 - accuracy: 0.9569 - val_loss: 1.4107 - val_accuracy: 0.5538
Epoch 4/30
15/15 [==============================] - 1s 72ms/step - loss: 0.0262 - accuracy: 0.9933 - val_loss: 1.7715 - val_accuracy: 0.5626
Epoch 5/30
15/15 [==============================] - 1s 72ms/step - loss: 0.0102 - accuracy: 0.9976 - val_loss: 2.0576 - val_accuracy: 0.5598
Epoch 6/30
15/15 [==============================] - 1s 72ms/step - loss: 0.0059 - accuracy: 0.9985 - val_loss: 2.2886 - val_accuracy: 0.5451
Epoch 7/30
15/15 [==============================] - 1s 74ms/step - loss: 0.0048 - accuracy: 0.9989 - val_loss: 2.3640 - val_accuracy: 0.5469
Epoch 8/30
1

In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
with open('/content/gdrive/My Drive/word_list.txt', 'r') as word:
    bow = word.read()

In [ ]:
with open('/content/gdrive/My Drive/tokenized_words.txt', 'w') as words:
    for word in word_index:
        words.write(word)  

with open('/content/gdrive/My Drive/tokenized_words.txt', 'r') as tok_words:
    tokenized_words = tok_words.read()

In [ ]:
bag_of_words = nlp(bow)
tweet_words = nlp(tokenized_words)

In [ ]:
similarity = round(tweet_words.similarity(bag_of_words)*100,2)